In [ ]:
import os
os.environ['CUDA_HOME'] = os.environ['CONDA_PREFIX']
os.environ['LIBRARY_PATH'] = os.environ['CONDA_PREFIX']+"/lib"
os.environ['LD_LIBRARY_PATH'] = os.environ['CONDA_PREFIX']+"/lib"

import dspy

In [ ]:
lm = dspy.HFModel(model="google/flan-t5-base", )
retriever = dspy.ColBERTv2(url='http://localhost:8001/api/search')
dspy.settings.configure(lm=lm, rm=retriever)

In [ ]:
class BasicQA(dspy.Signature):
    """Answer the question with a short answer."""

    question = dspy.InputField()
    answer = dspy.OutputField()

qa = dspy.Predict(BasicQA)

In [ ]:
ans = qa(question="What is Fringe?")
print(ans.answer)

In [ ]:
ans = qa(question="What is the name of the third episode of Fringe?")
print(ans.answer)

In [ ]:
class GenerateAnswerWithContext(dspy.Signature):
    """Answer the question with a short answer."""

    context = dspy.InputField(desc="may contain relevant facts to consider")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="short answer")


class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswerWithContext)
    
    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)
    
rag = RAG()

In [ ]:
ans = rag(question="What is Fringe?")
print(ans.answer)

In [ ]:
ans = rag(question="What is the name of the third episode of Fringe?")
print(ans.answer)

In [ ]:
ans = rag(question="What does Fringe Division do?")
print(ans.answer)

In [ ]:
ans = rag(question="Which episodes are directed by J. J. Abrams?")
print(ans.answer)